<a href="https://colab.research.google.com/github/salilp42/salilp42.github.io/blob/master/Deep_Autoencoder_PD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# First, we need to install the necessary packages
!pip install keras
!pip install tensorflow

# Import the necessary modules
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import regularizers

# Define the input shape and number of filters for the convolutional layers
input_shape = (256, 256, 1) # assuming the saccadic trajectories are 256x256 grayscale images
num_filters = 32

# Define the input layer
input_img = Input(shape=input_shape)

# Define the encoder layers
x = Conv2D(num_filters, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(num_filters, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Define the encoding layer with L1 regularization
encoding_layer = Dense(32, activation='relu', activity_regularizer=regularizers.l1(10e-5))(encoded)

# Define the decoder layers
x = Conv2D(num_filters, (3, 3), activation='relu', padding='same')(encoding_layer)
x = UpSampling2D((2, 2))(x)
x = Conv2D(num_filters, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

# Define the autoencoder model
autoencoder = Model(input_img, decoded)

# Train the autoencoder
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(X_train, X_train, epochs=50, batch_size=128, validation_data=(X_test, X_test))

# Get the encoder model
encoder = Model(input_img, encoding_layer)

# Use the encoder model to generate input for classifiers
encoded_imgs = encoder.predict(X_train)

# Now we can train a classifier using the encoded images and supervised labels
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(encoded_imgs, y_train)
